In [45]:
import pandas as pd
from sklearn import tree
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

In [3]:
ghost_train=pd.read_csv("D:/study/datascience/ghost/train.csv")

In [63]:
ghost_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 7 columns):
id               371 non-null int64
bone_length      371 non-null float64
rotting_flesh    371 non-null float64
hair_length      371 non-null float64
has_soul         371 non-null float64
color            371 non-null object
type             371 non-null object
dtypes: float64(4), int64(1), object(2)
memory usage: 20.4+ KB


In [64]:
ghost_train.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul
3,4,0.776652,0.508723,0.636766,0.884464,black,Ghoul
4,5,0.566117,0.875862,0.418594,0.636438,green,Ghost


In [65]:
ghost_train.describe()

,id,bone_length,rotting_flesh,hair_length,has_soul
count,371.000000,371.000000,371.000000,371.000000,371.000000
mean,443.676550,0.434160,0.506848,0.529114,0.471392
std,263.222489,0.132833,0.146358,0.169902,0.176129
min,0.000000,0.061032,0.095687,0.134600,0.009402
25%,205.500000,0.340006,0.414812,0.407428,0.348002
50%,458.000000,0.434891,0.501552,0.538642,0.466372
75%,678.500000,0.517223,0.603977,0.647244,0.600610
max,897.000000,0.817001,0.932466,1.000000,0.935721


In [66]:
ghost_train.corr()

,id,bone_length,rotting_flesh,hair_length,has_soul
id,1.000000,-0.062759,-0.007539,0.076475,0.100054
bone_length,-0.062759,1.000000,-0.041716,0.353881,0.381675
rotting_flesh,-0.007539,-0.041716,1.000000,-0.220353,-0.132051
hair_length,0.076475,0.353881,-0.220353,1.000000,0.474835
has_soul,0.100054,0.381675,-0.132051,0.474835,1.000000


In [224]:
import statistics 
statistics.stdev(ghost_train.bone_length)
statistics.stdev(ghost_train.rotting_flesh)
statistics.stdev(ghost_train.hair_length)
statistics.stdev(ghost_train.has_soul)

0.1761293396182109

In [67]:
x_train=pd.get_dummies(ghost_train.drop(['type','id'],1),columns=['color'])

In [47]:
def convert_soul(size):
    if(size >= 0.5): 
        return 1
    else: 
        return 0

In [114]:
x_train['soul1']=x_train.has_soul.map(convert_soul)
x_train=x_train.drop('has_soul',1)

In [68]:
x_train.head()

,bone_length,rotting_flesh,hair_length,has_soul,color_black,color_blood,color_blue,color_clear,color_green,color_white
0,0.354512,0.350839,0.465761,0.781142,0,0,0,1,0,0
1,0.575560,0.425868,0.531401,0.439899,0,0,0,0,1,0
2,0.467875,0.354330,0.811616,0.791225,1,0,0,0,0,0
3,0.776652,0.508723,0.636766,0.884464,1,0,0,0,0,0
4,0.566117,0.875862,0.418594,0.636438,0,0,0,0,1,0


In [69]:
y_train=ghost_train['type']

In [70]:
y_train.head()

0     Ghoul
1    Goblin
2     Ghoul
3     Ghoul
4     Ghost
Name: type, dtype: object

1. DecisionTree without Param

In [205]:
clf = tree.DecisionTreeClassifier(criterion="gini",random_state=0)
x = model_selection.cross_val_score(clf,x_train,y_train,cv=5)

In [206]:
x.mean()

0.6118622732321363

2. DecisionTree with GridSearchCV

In [207]:
dt = tree.DecisionTreeClassifier(random_state=0)
param_grid = {'max_depth':[5,8,10],'min_samples_split':[2,4,8,12,16],'criterion':['entropy','gini']}
dt_grid = model_selection.GridSearchCV(dt,param_grid)
dt_grid.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [5, 8, 10], 'min_samples_split': [2, 4, 8, 12, 16], 'criterion': ['entropy', 'gini']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [208]:
print(dt_grid.best_score_)
print(dt_grid.best_params_)

0.7142857142857143
{'criterion': 'gini', 'max_depth': 8, 'min_samples_split': 16}


3. RandomForest

In [71]:
rf_classifier = ensemble.RandomForestClassifier(random_state=0)
rf_grid = {'n_estimators':[10,20,30],'max_features':[4,5,6,7,8,9,10],'min_samples_split':[3,4,6]}
grid_rf_classifier = model_selection.GridSearchCV(rf_classifier,rf_grid,cv=10,n_jobs=3)
grid_rf_classifier.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'n_estimators': [10, 20, 30], 'max_features': [4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [3, 4, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [215]:
print(grid_rf_classifier.best_score_)
print(grid_rf_classifier.best_params_)

0.7169811320754716
{'max_features': 9, 'min_samples_split': 6, 'n_estimators': 30}


In [81]:
ghost_test=pd.read_csv("D:/study/datascience/ghost/test.csv")
x_test=pd.get_dummies(ghost_test.drop(['id'],1),columns=['color'])

In [82]:
ghost_test['output']=grid_rf_classifier.predict(x_test)

In [86]:
ghost_test[['id','output']].to_csv("D:/study/datascience/ghost/output.csv")

4. AdaBoost

In [216]:
ada_boost_estimator = ensemble.AdaBoostClassifier(random_state=0)
ada_boost_grid = {'n_estimators':[10,2,5,8,12,15,20],"learning_rate":[0.1,0.2,0.3,0.4]}
ada_boost_estimetor = model_selection.GridSearchCV(ada_boost_estimator,ada_boost_grid,cv=10,n_jobs=3)
ada_boost_estimetor.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=0),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'n_estimators': [10, 2, 5, 8, 12, 15, 20], 'learning_rate': [0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [217]:
print(ada_boost_estimetor.best_score_)
print(ada_boost_estimetor.best_params_)

0.7035040431266847
{'learning_rate': 0.4, 'n_estimators': 10}


In [13]:
ghost_train_data = pd.get_dummies(ghost_train.drop(['id'],1),columns=['color'])

In [17]:
train_data,train_test=model_selection.train_test_split(ghost_train_data,test_size=0.33)

In [18]:
print(train_data.info())
print(train_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248 entries, 208 to 337
Data columns (total 11 columns):
bone_length      248 non-null float64
rotting_flesh    248 non-null float64
hair_length      248 non-null float64
has_soul         248 non-null float64
type             248 non-null object
color_black      248 non-null uint8
color_blood      248 non-null uint8
color_blue       248 non-null uint8
color_clear      248 non-null uint8
color_green      248 non-null uint8
color_white      248 non-null uint8
dtypes: float64(4), object(1), uint8(6)
memory usage: 13.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 123 entries, 84 to 96
Data columns (total 11 columns):
bone_length      123 non-null float64
rotting_flesh    123 non-null float64
hair_length      123 non-null float64
has_soul         123 non-null float64
type             123 non-null object
color_black      123 non-null uint8
color_blood      123 non-null uint8
color_blue       123 non-null uint8
color_clear      1

In [19]:
x_train_data=train_data.drop('type',1)

In [20]:
y_train_data=train_data.type

In [24]:
rf_classifier_1 = ensemble.RandomForestClassifier(random_state=0,n_estimators=30,max_features=9,min_samples_split=6)
rf_classifier_1.fit(x_train_data,y_train_data)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=9, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [32]:
train_test['output']=rf_classifier_1.predict(train_test.drop('type',1))

ValueError: could not convert string to float: 'Ghost'

In [59]:
metrics.confusion_matrix(train_test.type,train_test.output,labels=train_test.type.unique())

array([[32,  1,  4],
       [ 1, 29,  7],
       [ 6, 16, 27]], dtype=int64)

In [56]:
metrics.accuracy_score(train_test.type, train_test.output)

0.7154471544715447

In [57]:
pd.crosstab(train_test.type,train_test.output)

output,Ghost,Ghoul,Goblin
type,,,
Ghost,32,1,4
Ghoul,1,29,7
Goblin,6,16,27


In [61]:
train_test.loc[train_test['type'] == 'Goblin']

,bone_length,rotting_flesh,hair_length,has_soul,type,color_black,color_blood,color_blue,color_clear,color_green,color_white,output
278,0.416577,0.458523,0.768333,0.654006,Goblin,0,0,0,1,0,0,Ghoul
234,0.284451,0.220268,0.357405,0.316073,Goblin,0,0,0,0,0,1,Goblin
301,0.577809,0.500339,0.756629,0.854949,Goblin,0,0,0,0,0,1,Ghoul
148,0.528987,0.496139,0.716997,0.332592,Goblin,0,0,0,0,0,1,Goblin
76,0.498336,0.374374,0.579926,0.511726,Goblin,0,0,0,1,0,0,Ghoul
231,0.275021,0.689975,0.586513,0.402749,Goblin,0,0,0,1,0,0,Ghost
315,0.483905,0.569917,0.634768,0.506433,Goblin,0,0,0,1,0,0,Ghoul
311,0.444006,0.424189,0.549613,0.447517,Goblin,0,0,0,0,1,0,Goblin
157,0.406880,0.464490,0.538146,0.600827,Goblin,0,0,0,0,0,1,Goblin
32,0.554510,0.630669,0.592484,0.615364,Goblin,0,0,0,0,0,1,Ghoul
